In [ ]:
# https://github.com/gcdart/MulticlassClassifier/blob/master/src/ml/LogisticRegression.java
# https://www.kaggle.com/c/lshtc/discussion/6911#38233 - preprocessing: multilabels comma should not have spaces
# https://www.kaggle.com/c/lshtc/discussion/14048 - dataset statistics
## reading the LWIKI, SWIKI dataset

In [ ]:
import sys

In [ ]:
sys.version

In [1]:
import os
# import numpy as np
# import pandas as pd

from tqdm import tqdm #always use this instead of `import tqdm`
# from sklearn.datasets import fetch_rcv1

In [2]:
import logging
from collections import OrderedDict

logging.basicConfig(level=logging.INFO)

In [3]:
from joblib import Memory
from sklearn.datasets import load_svmlight_file

In [4]:
file = open("swiki/data/dum_remapped.txt", "w+")

with open("swiki/data/dum.txt", "r") as f:
    for _, line in enumerate(tqdm(f)):
        instance = line.strip().split()
        labels = instance[0]
        print(labels)
        doc_dict = OrderedDict()
        temp_dict = {}
        temp_string = ''

        for pair in instance[1:]:
            feat = pair.split(":")
            if int(feat[0]) not in temp_dict:
                temp_dict[int(feat[0])] = int(feat[1])

        for key in sorted(temp_dict.keys()):
            doc_dict[key] = temp_dict[key]

        for feat, tf in doc_dict.items():
            temp_string = temp_string + "{}:{} ".format(feat, tf)        
        file.write("{} {}\n".format(labels, temp_string))
file.close()

0it [00:00, ?it/s]

0,1
0
0
0
0
0
0
0
0


9it [00:00, 598.41it/s]


In [4]:
def preprocess_libsvm(input_file, output_file):
    
    file = open(output_file, "w")

    with open(input_file, "r") as f:
        the_file = ''
        for i, line in enumerate(tqdm(f)):
            instance = line.strip().split()
            labels = instance[0]
            doc_dict = OrderedDict()
            temp_dict = {}
            temp_string = ''

            for pair in instance[1:]:
                feat = pair.split(":")
                if int(feat[0]) not in temp_dict:
                    temp_dict[int(feat[0])] = int(feat[1])
                else:
                    print(feat[0])

            for key in sorted(temp_dict.keys()):
                doc_dict[key] = temp_dict[key]

            for feat, tf in doc_dict.items():
                temp_string = temp_string + "{}:{} ".format(feat, tf)        
            temp_string = labels + " " + temp_string
            the_file = the_file + temp_string + "\n"

    file.write(the_file)
    file.close()
    
    return output_file

In [27]:
mem = Memory("./mycache")
@mem.cache
def get_data(filename):
    
    try:
        print("try")
        data = load_svmlight_file(filename, multilabel=True)
    
    except:
        # when there is an error since the dataset isn't in the correct libsvm format. *sigh*
        # so we need to ValueError: Feature indices in SVMlight/LibSVM data file should be sorted and unique get rid of that
        # *SIGH*
                
        temp = filename.split(".")
        outfile = temp[0] + "_remapped." + temp[1]
        
        if not os.path.isfile(outfile):
            logging.warning("Remapping data to LibSVM format...")
            f = preprocess_libsvm(filename, outfile)
        else:
            logging.warning("Using already remapped data...")
            f = outfile
            
        data = load_svmlight_file(f, multilabel=True)
        
    return data[0], data[1]

In [38]:
fff = "..\swiki\data\dummy.txt"

In [41]:
fe, ex = os.path.splitext(fff)

In [43]:
ex

'.txt'

In [28]:
# x contains all the feat:value data
# y contains all the labels (per line)
# x, y = get_data("swiki/data/train.txt")
# a, b = get_data("swiki/data/test_remapped.txt")
x, y = get_data("swiki/data/dummy.txt")

________________________________________________________________________________
[Memory] Calling __main__-C%3A-Users-harshasivajit-Documents-master-ai-rr13-__ipython-input__.get_data...
get_data('swiki/data/dummy.txt')
try


_________________________________________________________get_data - 0.0s, 0.0min


In [22]:
x.shape #num documents X num features (training set)

(11, 2076509)

In [23]:
print("{} size".format(x.shape))

(11, 2076509) size


In [24]:
y

[(85048.0, 175937.0, 189243.0),
 (302422.0, 378629.0),
 (13402.0, 33692.0, 393382.0),
 (130762.0,),
 (27512.0, 157031.0, 352578.0, 395447.0),
 (276114.0, 390846.0, 395447.0),
 (14661.0, 71999.0, 131368.0, 188756.0, 292915.0),
 (106615.0, 228092.0),
 (228506.0, 420589.0),
 (207086.0, 307963.0),
 (119558.0, 159586.0)]

In [ ]:
# lets see how many unique labels are there and if it verifies with the paper
# ooooh i only get the leaf labels with this. or like atleast when i add it to a set.
# how do i get class labels? what are class labels first!
leaf_labels = set()
class_labels = []
count_y = []
for i in y:
    count_y.append(len(i))
    for j in i:
        leaf_labels.add(j)
        class_labels.append(j)

In [ ]:
print(len(leaf_labels))

In [ ]:
# these leaf labels range from 1 - 445729 (training set) this number is the max label id
list(leaf_labels)[325055] #[0]

In [ ]:
# this is just the plain brute force counting of all labels for all documents.
# same as count_y
len((class_labels))

In [ ]:
# avg num_labels per instance(training doc)
sum(count_y)/len(y)

### test set

In [ ]:
a.shape
# len(b)

In [ ]:
leaf_t = set()
class_t = []

for i in b:
    for j in i:
        leaf_t.add(j)
        class_t.append(j)

In [ ]:
len(class_t)

In [ ]:
904334/2 # <- num of labels are doubled basically due to the 0 in every document

### hierarchy.txt

In [ ]:
def read_hier(filename):
    
    unique_h = set()
    
    with open(filename, "r") as f:
        for i, line in enumerate(f):
            words = line.strip().split()
            for w in words:
                unique_h.add(int(w))

    return unique_h

In [ ]:
# u = read_hier("hierarchy/hierarchy.txt")
u = read_hier("swiki/data/cat_hier.txt")

In [ ]:
len(u)

In [ ]:
u